# Higuchi fractal dimension on real and synthetic EEG-like signals

In this notebook, we will:

1. implement the Higuchi fractal dimension (HFD) for 1D time series,  
2. load a real EEG segment from the CHB-MIT dataset and estimate its HFD,  
3. build several synthetic EEG-like signals (pure sinusoid, sinusoid + noise, band-limited noise, noise + bursts), compute their HFD, and compare them across multiple realizations, and  
4. construct a slow–fast bursting oscillator and compare its HFD to a regular oscillator.

The purpose is to connect the mathematical idea of fractal dimension to concrete neural-like signals and observe how HFD changes across different levels of complexity.


## Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import mne
from scipy.signal import butter, filtfilt

rng = np.random.default_rng(42)


## Implementing the Higuchi fractal dimension

Higuchi's method estimates the fractal dimension of a time series by constructing subsampled curves at different scales `k`, computing their effective lengths, and then regressing `\log L(k)` against `\log(1/k)`. Below is a standard implementation of the Higuchi fractal dimension for a 1D NumPy array.


In [ ]:
def higuchi_fd(x, k_max=8):
    """
    compute higuchi fractal dimension of a 1d time series x.
    """
    x = np.asarray(x, dtype=float)
    n = x.size
    if n < 2:
        raise ValueError("time series must have length at least 2")

    Lk = []
    k_values = []

    for k in range(1, k_max + 1):
        Lm = []
        for m in range(k):
            idx = m + np.arange(0, int(np.floor((n - m - 1) / k)) + 1) * k
            if idx.size < 2:
                continue
            xm = x[idx]
            diff_sum = np.sum(np.abs(np.diff(xm)))
            norm = (n - 1) / (idx.size * k)
            Lm.append(diff_sum * norm)
        if len(Lm) > 0:
            Lk.append(np.mean(Lm))
            k_values.append(k)

    Lk = np.array(Lk)
    k_values = np.array(k_values)
    slope, intercept = np.polyfit(np.log(1.0 / k_values), np.log(Lk), 1)
    return slope


## Sanity check on simple signals

As a quick sanity check, we compute the Higuchi fractal dimension for two simple signals:

- a pure sinusoid, and  
- white noise.  

We expect the sinusoid to have a lower fractal dimension than the noise, because it is more regular.


In [ ]:
sfreq = 256.0
duration = 10.0
t = np.arange(0, duration, 1.0/sfreq)

x_sine = np.sin(2*np.pi*10*t)
x_noise = rng.normal(0,1,size=t.size)

print("hfd sine:", higuchi_fd(x_sine))
print("hfd noise:", higuchi_fd(x_noise))

fig,axes = plt.subplots(2,1,figsize=(10,5),sharex=True)
axes[0].plot(t,x_sine); axes[0].set_title("pure sinusoid")
axes[1].plot(t,x_noise); axes[1].set_title("white noise")
plt.tight_layout(); plt.show()


## Loading a real EEG segment from the CHB-MIT dataset

Next, we load an EDF file from the CHB-MIT scalp EEG database using MNE. You will need to update the file path to point to a local CHB-MIT EDF file on your machine.


In [ ]:
edf_path = "/path/to/chbmit/chb01/chb01_01.edf"

raw = mne.io.read_raw_edf(edf_path, preload=True, verbose="warning")
print(raw)
print(raw.ch_names)


### Selecting a channel and computing HFD

Here we:

1. select a single EEG channel,  
2. extract a 10-second segment,  
3. plot the segment, and  
4. compute its Higuchi fractal dimension.


In [ ]:
channel_name = raw.ch_names[0]
raw_chan = raw.copy().pick_channels([channel_name])

tmin, tmax = 0.0, 10.0
raw_seg = raw_chan.copy().crop(tmin=tmin, tmax=tmax)

data, times = raw_seg.get_data(return_times=True)
eeg_segment = data[0]

plt.figure(figsize=(10,3))
plt.plot(times, eeg_segment)
plt.title(f"eeg segment ({channel_name})")
plt.xlabel("time (s)")
plt.ylabel("amplitude (µv, approx)")
plt.tight_layout()
plt.show()

print("hfd eeg:", higuchi_fd(eeg_segment))


## Synthetic EEG-like signals

To understand how HFD behaves on different kinds of signals, we construct four synthetic EEG-like signals:

1. a pure sinusoid,  
2. a sinusoid plus noise,  
3. band-limited noise, and  
4. noise with intermittent bursts.  

For each type, we generate multiple trials and compute HFD for each trial.


In [ ]:
def band_limited_noise(n_samples, sfreq, fmin, fmax, rng):
    x = rng.normal(0,1,size=n_samples)
    nyq = 0.5*sfreq
    b,a = butter(4,[fmin/nyq,fmax/nyq],btype="band")
    return filtfilt(b,a,x)

def bursty_signal(n_samples, sfreq, base_std=0.4, burst_std=3.0, n_bursts=5, burst_len=0.2, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    x = rng.normal(0, base_std, size=n_samples)
    L = int(burst_len*sfreq)
    for _ in range(n_bursts):
        start = rng.integers(0, n_samples-L)
        x[start:start+L] += rng.normal(0, burst_std, size=L)
    return x


In [ ]:
sfreq_syn = 256
t_syn = np.arange(0,10,1/sfreq_syn)
n = t_syn.size
n_trials = 20

fd_results = {"sine":[],"sine+noise":[],"band":[],"bursty":[]}

for _ in range(n_trials):
    sigA = np.sin(2*np.pi*10*t_syn)
    sigB = sigA + rng.normal(0,0.3,size=n)
    sigC = band_limited_noise(n, sfreq_syn, 8,12, rng)
    sigD = bursty_signal(n, sfreq_syn, rng=rng)

    fd_results["sine"].append(higuchi_fd(sigA))
    fd_results["sine+noise"].append(higuchi_fd(sigB))
    fd_results["band"].append(higuchi_fd(sigC))
    fd_results["bursty"].append(higuchi_fd(sigD))

for k,v in fd_results.items():
    print(k, np.mean(v), np.std(v))


### Example time series from each signal class

Here we plot one example realization from each of the four synthetic signal classes so that we can visually compare their structure.


In [ ]:
sigA = np.sin(2*np.pi*10*t_syn)
sigB = sigA + rng.normal(0,0.3,size=n)
sigC = band_limited_noise(n, sfreq_syn, 8,12, rng)
sigD = bursty_signal(n, sfreq_syn, rng=rng)

fig,ax = plt.subplots(4,1,figsize=(10,8),sharex=True)
for a,s,title in zip(ax,[sigA,sigB,sigC,sigD],
                     ["sine","sine+noise","band-limited noise","bursty noise"]):
    a.plot(t_syn,s); a.set_title(title)
plt.tight_layout(); plt.show()


### HFD comparison across synthetic signals

We now summarize the HFD values across all trials for each synthetic signal type and plot the mean and standard deviation for each group.


In [ ]:
labels = list(fd_results.keys())
means = [np.mean(fd_results[k]) for k in labels]
stds = [np.std(fd_results[k]) for k in labels]

x = np.arange(len(labels))
plt.figure(figsize=(7,4))
plt.bar(x,means,yerr=stds,capsize=5)
plt.xticks(x,labels)
plt.ylabel("higuchi fractal dimension")
plt.title("hfd across synthetic eeg-like signals")
plt.tight_layout()
plt.show()


## Slow–fast bursting oscillator

Finally, we compare a regular oscillator to a slow–fast bursting oscillator. The bursting oscillator is created by modulating a fast sinusoid with a slow envelope, which introduces two time scales and richer dynamics.


In [ ]:
sfreq_b = 256
t_b = np.arange(0,20,1/sfreq_b)

osc_regular = np.sin(2*np.pi*10*t_b)
envelope = 0.5*(1 + np.sin(2*np.pi*0.5*t_b))
osc_burst = envelope * np.sin(2*np.pi*10*t_b)

osc_regular += rng.normal(0,0.1,size=t_b.size)
osc_burst += rng.normal(0,0.1,size=t_b.size)

print("hfd regular:", higuchi_fd(osc_regular))
print("hfd burst:", higuchi_fd(osc_burst))

plt.figure(figsize=(10,5))
plt.subplot(2,1,1); plt.plot(t_b,osc_regular); plt.title("regular oscillator")
plt.subplot(2,1,2); plt.plot(t_b,osc_burst); plt.title("bursting oscillator")
plt.tight_layout(); plt.show()


## Summary

In this notebook, we implemented the Higuchi fractal dimension, applied it to a real EEG segment from the CHB-MIT dataset, generated several classes of synthetic EEG-like signals, compared their fractal dimensions, and examined how bursting dynamics affect HFD. Together, these examples illustrate how fractal methods can be used to quantify aspects of neural signal complexity in both theoretical and applied settings.
